In [43]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from sklearn.cluster import KMeans

pd.options.mode.chained_assignment = None

Now we are downloading the list of postal codes and transform into a dataframe.

In [4]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())
My_table=soup.find('table')

column_names = ['Postal Code', 'Borough', 'Neighbourhood']
list_of_postal_code = pd.DataFrame(columns=column_names)

for row in My_table.findAll('tr'):
    for cell_td in row.findAll('td'):
        cell_postal_code=cell_td.findAll('b')
        cell_borough=cell_td.findAll('a')
        
        if len(cell_borough) > 1:
            list_of_neighbourhood = ''
            #list_of_postal_code = list_of_postal_code.append({'Postal Code': cell_postal_code[0].find(text=True),'Borough': cell_borough[0].find(text=True)},ignore_index=True)
            for neighbourhood in cell_borough:
                if neighbourhood.find(text=True) != cell_borough[0].find(text=True):
                    if list_of_neighbourhood == '':
                        list_of_neighbourhood = neighbourhood.find(text=True)
                    else:
                        list_of_neighbourhood = list_of_neighbourhood + ', ' +(neighbourhood.find(text=True))
            #print (list_of_neighbourhood)
            list_of_postal_code = list_of_postal_code.append({'Postal Code': cell_postal_code[0].find(text=True),'Borough': cell_borough[0].find(text=True),'Neighbourhood': list_of_neighbourhood},ignore_index=True)

list_of_postal_code.shape

(88, 3)

Downloading coordinates

In [5]:
coordinates_df = pd.read_csv('http://cocl.us/Geospatial_data',header=0)
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Setting coordinates of postal codes

In [7]:
#Add 2 new empty columns to existing dataframe
header_list = ['Postal Code', 'Borough', 'Neighbourhood','Latitude','Longitude']
list_of_postal_code = list_of_postal_code.reindex(columns = header_list)

for ind_postal_code in list_of_postal_code.index: 
    for ind_coordinate in coordinates_df.index:
        if coordinates_df['Postal Code'][ind_coordinate] == list_of_postal_code['Postal Code'][ind_postal_code]:
            list_of_postal_code['Latitude'][ind_postal_code] = coordinates_df['Latitude'][ind_coordinate]
            list_of_postal_code['Longitude'][ind_postal_code] = coordinates_df['Longitude'][ind_coordinate]

list_of_postal_code
        #print(list_of_postal_code['Postal Code'][ind_postal_code])

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
83,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
84,M8X,Etobicoke,"The Kingsway, Old Mill",43.653654,-79.506944
85,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
86,M8Y,Etobicoke,"Old Mill, Sunnylea, Humber Bay, Mimico, The Qu...",43.636258,-79.498509


Explore

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    CLIENT_ID = 'MOO2HCYSDVVR3KY41DN3T4OX14TUA54EW5BQNQCEY4DU20EM' # your Foursquare ID
    CLIENT_SECRET = 'ITFA224F1HXQRPYS5V1CKAP1ZH4CABDVLYTG1UBLBXIPWJ1H' # your Foursquare Secret
    VERSION = '20200324'
    LIMIT = 30
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
toronto_borough = list_of_postal_code[list_of_postal_code['Borough'].str.contains("Toronto")] 
toronto_venues = getNearbyVenues(names=toronto_borough['Neighbourhood'],
                                   latitudes=toronto_borough['Latitude'],
                                   longitudes=toronto_borough['Longitude'])
toronto_venues.shape
#toronto_venues.head()

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Bay Street
Richmond, King
Harbourfront, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


(412, 7)

Create Map

In [10]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_borough['Latitude'], toronto_borough['Longitude'], toronto_borough['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Cluster neigbourhood

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
toronto_onehot.shape

(412, 141)

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.033333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333
3,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,...,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
5,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,...,0.033333,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Harbourfront, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.066667,0.033333,0.000000
8,"Regent Park, Harbourfront",0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Richmond, King",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000


In [25]:
toronto_grouped.shape

(15, 141)

In [26]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Street----
                 venue  freq
0          Coffee Shop  0.20
1       Ice Cream Shop  0.07
2   Italian Restaurant  0.07
3  Japanese Restaurant  0.07
4   Chinese Restaurant  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3             Plane  0.06
4           Airport  0.06


----Church and Wellesley----
                venue  freq
0         Coffee Shop  0.07
1           Wine Shop  0.03
2        Dance Studio  0.03
3  Salon / Barbershop  0.03
4          Restaurant  0.03


----Commerce Court, Victoria Hotel----
                 venue  freq
0                 Café  0.17
1          Coffee Shop  0.10
2           Restaurant  0.10
3  Japanese Restaurant  0.07
4            Gastropub  0.07


----First Canadian Place, Underground city----
                venue  freq
0                Café  0.13
1          Restaurant  0.10
2      

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Street,Coffee Shop,Chinese Restaurant,Japanese Restaurant,Italian Restaurant,Ice Cream Shop,Spa,Modern European Restaurant,Miscellaneous Shop,Gastropub,Café
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Sculpture Garden,Plane,Bar,Rental Car Location,Harbor / Marina,Boutique
2,Church and Wellesley,Coffee Shop,Wine Shop,Ramen Restaurant,Indian Restaurant,Dance Studio,Salon / Barbershop,Bookstore,Restaurant,Creperie,Pub
3,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Restaurant,Japanese Restaurant,Deli / Bodega,Gastropub,Gym,Steakhouse,Gym / Fitness Center,Beer Bar
4,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Gastropub,Seafood Restaurant,Gluten-free Restaurant,Bakery,General Travel,Pizza Place,Deli / Bodega


In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 3, 2, 0, 0, 2, 2, 4, 2, 2])

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_borough

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Park,Mexican Restaurant,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio,Spa
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Burrito Place,Plaza,Shopping Mall,Sandwich Place,Restaurant,Ramen Restaurant,Japanese Restaurant
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Gastropub,Italian Restaurant,Café,Japanese Restaurant,Restaurant,Hotel,Hostel,Beer Bar,Creperie
21,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383,0,Coffee Shop,Chinese Restaurant,Japanese Restaurant,Italian Restaurant,Ice Cream Shop,Spa,Modern European Restaurant,Miscellaneous Shop,Gastropub,Café
26,M5H,Downtown Toronto,"Richmond, King",43.650571,-79.384568,2,Hotel,Seafood Restaurant,Café,Asian Restaurant,Steakhouse,Restaurant,Pizza Place,Lounge,Monument / Landmark,Colombian Restaurant


In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters